In [1]:
import pandas as pd

# Add genotype for SV

In [2]:
def add_genotype(infile, outfile):
    dat = pd.read_csv(infile, sep="\t")
    if False: # loose
        min_cell = 1
        min_read = 2
        foldchange = 2
    else: # stringent
        min_cell = 2
        min_read = 4
        foldchange = 3
    for hp in ["HP1", "HP2"]:
        values = dat[["AgreeRead_%s" % hp, "AgreeCell_%s" % hp, "DisagreeRead_%s" % hp, "DisagreeCell_%s" % hp]].values
        ts = []
        for agree_read, agree_cell, disagree_read, disagree_cell in values:
            t = "Uncertain" # Exist, NotExist
            if agree_cell >= min_cell and agree_cell >= disagree_cell * foldchange \
                and agree_read >= min_read and agree_read >= disagree_read * foldchange:
                t = "Exist"
            if disagree_cell >= min_cell and disagree_cell >= agree_cell * foldchange \
                and disagree_read >= min_read and disagree_read >= agree_read * foldchange:
                assert t != "Exist"
                t = "NotExist"
            ts.append(t)
        dat["Existence_%s" % hp] = ts
    gts = []
    for v1, v2 in dat[["Existence_HP1", "Existence_HP2"]].values:
        gt = "."
        if v1 == "Exist":
            if v2 == "Exist":
                gt = "1|1"
            elif v2 == "NotExist":
                gt = "1|0"
        elif v1 == "NotExist":
            if v2 == "Exist":
                gt = "0|1"
        gts.append(gt)
    dat["GenoType"] = gts
    dat.to_csv(outfile, sep="\t", index=False)

ncells = [50, 75, 100, 125, 150, 200, 250, 300, 350]
for ncell in ncells:
    infile = "../../4_NanoStrandSeq_Phasing/results/HG001_Cell_%d/sv/quantify_lite.tsv" % ncell
    outfile = "results/quant_phased_svs.HG001_Cell_%d.tsv" % ncell
    add_genotype(infile, outfile)